# Pytest

[Pytest](https://docs.pytest.org/en/latest/contents.html) has rapidly established itself as the standard python testing framework. Its power lies in its simplicity, which makes it super easy to write tests and to run them. Tests can be as simple as the ones that we've already seen, so it is perfect for small projects. However, hidden behind the simple exterior, `pytest` provides a great deal of power and flexibility, meaning that it scales well to large and complex projects.

In this section we will learn how automatically find and run tests using `pytest` and how to query and interpret the output.

## Running tests

Pytest comes with a command-line tool called, unsurprisingly, `pytest`. (With older versions of python this was called `py.test`.) When run, `pytest` searches for tests in all directories and files below the current directory, collects the tests together, then runs them. Pytest uses name matching to locate the tests. Valid names start or end with `test`, e.g.

```bash
test_example.py    # We will use this as our standard naming convention.
example_test.py
```

(Note that this naming convention applies to test functions, as well as files.)

You can specify one or more paths and `pytest` will only look for tests in those paths, e.g.

```bash
pytest /path/to/my/awesome/module
```

You can also specify the name of a file (or files) containing test functions, e.g.

```bash
pytest /path/to/my/awesome/module/test/tests.py
```

When writing a python module it is good practice to set up a directory structure in order to keep things tidy. Throughout this course we will use the following:

```bash
mypkg/
    __init__.py
    whizz.py
    bang.py
    test/
        __init__.py
        test_whizz.py
        test_bang.py
```

Here the `__init__.py` files makes python aware that the directories should be treated as modules. Assuming we're in the top level directory (where out notebooks reside) this allows us to do the following:

```python
import mypkg
```

Let's dive in and run some tests using `mymodule` that was introduced in the previous section.

In [20]:
!pytest mymodule

============================= test session starts ==============================
platform linux -- Python 3.6.3, pytest-3.3.0, py-1.5.0, pluggy-0.6.0
rootdir: /home/lester/Code/siremol.org/chryswoods.com/python_and_data/testing, inifile:
collected 17 items                                                             

mymodule/test/test_errors.py .s                                          [ 11%]
mymodule/test/test_mymath.py x.......s..F...                             [100%]

=================================== FAILURES ===================================
____________________________ test_greaterThan[3-7] _____________________________

x = 3, y = 7

    @pytest.mark.parametrize("x, y",
                            [(108, 56),
                             (-64, -333),
                             (3, 7),
                             (74, 15)])
    def test_greaterThan(x, y):
        """ Test the greaterThan function. """
    
        if x > y:
            assert greaterThan(x, y)
        

What just happened?

Well, `pytest` searched within the `mymodule` directory and collected a total of 17 tests. These tests were spread accross two files:

```bash
mymodule/test/test_errors.py
mymodule/test/test_mymath.py
```

The tests were then run and a single failure was reported `test_greaterThan`. This test failed when asserting that 3 shouldn't be greater than 7.

```python
E           assert not True
E            +  where True = greaterThan(3, 7)
```

What are all the cryptic symbols next to the name of each test file?

```bash
mymodule/test/test_mymath.py x.......s...F....                           [100%]
```

To get more detailed information about each test, we can run `pytest` using the _verbose_ flag.

In [21]:
!pytest mymodule -v

============================= test session starts ==============================
platform linux -- Python 3.6.3, pytest-3.3.0, py-1.5.0, pluggy-0.6.0 -- /usr/bin/python
cachedir: .cache
rootdir: /home/lester/Code/siremol.org/chryswoods.com/python_and_data/testing, inifile:
collected 17 items                                                             

mymodule/test/test_errors.py::test_indexError PASSED                     [  5%]
mymodule/test/test_errors.py::test_BSoD SKIPPED                          [ 11%]
mymodule/test/test_mymath.py::test_add xfail                             [ 17%]
mymodule/test/test_mymath.py::test_sub[1-2--1] PASSED                    [ 23%]
mymodule/test/test_mymath.py::test_sub[7-3-4] PASSED                     [ 29%]
mymodule/test/test_mymath.py::test_sub[21-58--37] PASSED                 [ 35%]
mymodule/test/test_mymath.py::test_mul[3-1] PASSED                       [ 41%]
mymodule/test/test_mymath.py::test_mul[3-2] PASSED                       [ 47%]
mymod

We now have more detailed information about each test. Matching up the output with the symbols we can see that:

```
. = PASSED
s = SKIPPED
F = FAILED
x = xfail
```

What does `xfail` mean, and why were some tests skipped? Also, note that some tests were run mutliple times, e.g. `test_sub` appears 3 times.

What do the numbers in square brackets mean? Looking at the report for the test that failed, we can see that the square brackets show the arguments to the test functions, e.g. `x=3` and `y=7`.

To _report_ more information on tests that were recorded as `SKIPPED`, or `xfail`, we can run `pytest` as follows.

In [22]:
!pytest mymodule -rsx

============================= test session starts ==============================
platform linux -- Python 3.6.3, pytest-3.3.0, py-1.5.0, pluggy-0.6.0
rootdir: /home/lester/Code/siremol.org/chryswoods.com/python_and_data/testing, inifile:
collected 17 items                                                             

mymodule/test/test_errors.py .s                                          [ 11%]
mymodule/test/test_mymath.py x.......s..F...                             [100%]
=========================== short test summary info ============================
SKIP [1] mymodule/test/test_errors.py:12: Only runs on Windows.
SKIP [1] mymodule/test/test_mymath.py:30: Not yet implemented.
XFAIL mymodule/test/test_mymath.py::test_add
  Broken code. Working on a fix.

=================================== FAILURES ===================================
____________________________ test_greaterThan[3-7] _____________________________

x = 3, y = 7

    @pytest.mark.parametrize("x, y",
                    

We now have additional information about these tests. One test was skipped because it is only valid on the Windows operating system, another because it was testing functionality that hasn't been implemented yet.

By now, you might have guessed that `xfail` means _expected to fail_. You can see that the `test_add` was expected to fail with the reason "Broken code. Working on a fix." This was the function that we found to be broken in the previous section.

Let's switch to the terminal and try to work out why the test was failing. Since we know that `test_greaterThan` was the culprit, let's look at the `greaterThan` function in `mymodule/mymodule.py` to see what could be going wrong.

Well, that was silly. Having fixed the bug, let's confirm that our tests now pass. Let's also run `pytest` in _quiet_ mode. Here we only get a minimal output showing the progress of the tests and reports for any failures (so hopefully none!).

In [24]:
!pytest mymodule -q

.sx.......s......                                                        [100%]
14 passed, 2 skipped, 1 xfailed in 2.76 seconds


# Exercises

More details on how to invoke `pytest` can be found [here](https://docs.pytest.org/en/latest/usage.html).

#### Exercise 1

Only run the `test_mul` unit test. This test is in the file `mymodule/test/test_mymath.py`

In [19]:
!pytest mymodule/test/test_mymath.py::test_mul

============================= test session starts ==============================
platform linux -- Python 3.6.3, pytest-3.3.0, py-1.5.0, pluggy-0.6.0
rootdir: /home/lester/Code/siremol.org/chryswoods.com/python_and_data/testing, inifile:
collected 4 items                                                              

mymodule/test/test_mymath.py ....                                        [100%]

=========================== 4 passed in 0.01 seconds ===========================


#### Exercise 2

Which are the three slowest tests?

In [25]:
!pytest mymodule --durations=3

============================= test session starts ==============================
platform linux -- Python 3.6.3, pytest-3.3.0, py-1.5.0, pluggy-0.6.0
rootdir: /home/lester/Code/siremol.org/chryswoods.com/python_and_data/testing, inifile:
collected 17 items                                                             

mymodule/test/test_errors.py .s                                          [ 11%]
mymodule/test/test_mymath.py x.......s......                             [100%]

=========================== slowest 3 test durations ===========================
2.75s call     mymodule/test/test_mymath.py::test_bigSum
0.00s setup    mymodule/test/test_errors.py::test_indexError
0.00s setup    mymodule/test/test_mymath.py::test_sub[7-3-4]
=============== 14 passed, 2 skipped, 1 xfailed in 2.78 seconds ================
